In [1]:
ref='nb121-' 

import pandas as pd
import h5py
import numpy as np

import datetime
import copy
import os


In [2]:
def convert_time_to_unix(dataframe, columns_to_convert):
    """ A function that converts columns specified by 'columns_to_convert' in the 'dataframe' into epoch time and returns the converted dataframe"""
    for col in columns_to_convert:
        dataframe[col]=dataframe[col].apply(lambda x: pd.Timestamp(x).timestamp())
    return dataframe

In [3]:
#derived data
annotation=pd.read_csv("../data/derived_data/annotation.csv")
#section_speed=pd.read_csv("../data/derived_data/section-speed.csv")
#steps=pd.read_csv("../data/derived_data/steps.csv")
wheel=pd.read_csv("../data/derived_data/wheel.csv")

#raw data
sync=pd.read_csv("../data/raw_data/sync.csv")
raw_file="../data/raw_data/raw.h5"
raw=h5py.File(raw_file,"r")

In [4]:
# Derived data
annotation.head()

,SubjectId,RunId,Annotation,AnnotationType,Comment,start,end
0,52,1,Sync,Event,NaN,2018-05-30 17:10:53.657,2018-05-30 17:10:55.947
1,52,1,hallway forward,Section,NaN,2018-05-30 17:11:56.120,2018-05-30 17:12:45.216
2,52,1,wheel behind,Event,NaN,2018-05-30 17:12:33.138,2018-05-30 17:12:38.154
3,52,1,wheel behind,Event,NaN,2018-05-30 17:12:56.175,2018-05-30 17:13:00.165
4,52,1,pause,Event,NaN,2018-05-30 17:13:07.084,2018-05-30 17:13:33.564


In [5]:
#Derived data
wheel.head()

,SubjectId,RunId,speed,start,end
0,52,1,0.133031,2018-05-30 17:11:52,2018-05-30 17:11:53
1,52,1,0.328511,2018-05-30 17:11:53,2018-05-30 17:11:54
2,52,1,0.462072,2018-05-30 17:11:54,2018-05-30 17:11:55
3,52,1,0.516548,2018-05-30 17:11:55,2018-05-30 17:11:56
4,52,1,0.591766,2018-05-30 17:11:56,2018-05-30 17:11:57


In [6]:
sync.head()

,SubjectId,RunId,wheel.tap.idx,subject.delta,comment,subject.path,wheel.path,subject.start,subject.end,subject.duration,wheel.start,wheel.end,wheel.duration,subject.belt.id,wheel.belt.id,video.delta,video.start
0,52,1,1898,1231,NaN,subject/052-1/5B0EDB12,wheel/052-1/5B0EDB06,2018-05-30 17:10:42,2018-05-30 17:23:56.15,794.15,2018-05-30 17:10:30,2018-05-30 17:22:05.00,695.00,10477,11080,-14.2924,2018-05-30 17:10:34.677
1,52,2,777,-1093,NaN,subject/052-2/5B0EDD73,wheel/052-2/5B0EDD7D,2018-05-30 17:20:51,2018-05-30 17:32:26.05,695.06,2018-05-30 17:21:01,2018-05-30 17:30:54.98,593.98,11049,11084,-32.9262,2018-05-30 17:20:35.833
2,78,1,2344,829,NaN,subject/078-1/5AF1C962,wheel/078-1/5AF1C95B,2018-05-08 15:59:30,2018-05-08 16:09:54.48,624.48,2018-05-08 15:59:23,2018-05-08 16:10:57.35,694.36,11080,10646,-13.7631,2018-05-08 15:59:32.666
3,78,2,1576,708,NaN,subject/078-2/5AF1CCBB,wheel/078-2/5AF1CCB4,2018-05-08 16:13:47,2018-05-08 16:19:22.68,335.68,2018-05-08 16:13:40,2018-05-08 16:20:48.59,428.60,10493,11084,-50.5836,2018-05-08 16:13:05.166
4,145,1,1450,-703,NaN,subject/145-1/5AF4004E,wheel/145-1/5AF40055,2018-05-10 08:18:22,2018-05-10 08:31:35.25,793.26,2018-05-10 08:18:29,2018-05-10 08:30:38.96,729.96,10477,10646,-28.7585,2018-05-10 08:18:14.731


In [7]:
# Print sample of accls and speed raw data as fetched form hdf diles
select_row=0

row=sync.iloc[select_row]
print('Printing sample raw data for both subject and wheel sensors for subject {} and run {}'.format(row['SubjectId'], row['RunId']))
print('\n')

# Fetch subject sensor raw data (accls) from raw.h5 
path_to_subject=row["subject.path"]
subject_data=raw.get(path_to_subject)
print('Path to subject sensor raw data {}'.format(path_to_subject))
subject_data=list(subject_data)
subject_data=np.array(subject_data).T
print('No. of sample points: {}'.format(len(subject_data)))
print(subject_data)

print('\n')

# Fetch wheel sensor raw data (accls) from raw.h5
path_to_wheel=row["wheel.path"]
wheel_data=raw.get(path_to_wheel)
print('Path to wheel sensor raw data {}'.format(path_to_wheel))
wheel_data=list(wheel_data)
wheel_data=np.array(wheel_data).T
print('No. of sample points: {}'.format(len(wheel_data)))
print(wheel_data)



Printing sample raw data for both subject and wheel sensors for subject 52 and run 1


Path to subject sensor raw data subject/052-1/5B0EDB12
No. of sample points: 79415
[[-34 -12 330   1]
 [-36 -13 329   1]
 [-35 -10 333   1]
 ...
 [324  -3   8   0]
 [327   0   4   0]
 [336   2  18   0]]


Path to wheel sensor raw data wheel/052-1/5B0EDB06
No. of sample points: 69500
[[-89  32 290   1]
 [-85  28 297   1]
 [-79  34 315   1]
 ...
 [349  11   6   0]
 [324   3   2   0]
 [334   8  10   0]]


In [8]:
# Convert datetime to unix time stamp for convenience 

wheel=convert_time_to_unix(wheel,['start','end'])
wheel['mid']=wheel['start']+0.5

sync=convert_time_to_unix(sync,['subject.start','subject.end','wheel.start','wheel.end','video.start'])
sync.to_csv('sync_temp.csv')
 
annotation=convert_time_to_unix(annotation,['start','end'])
annotation.to_csv('annotation_temp.csv')

In [9]:
# 1) Get timestamp of tap from the wheel timestamps 
#    (will have to generate a small time-series of the start of the test based on wheel start time and 100hz sampling rate)



subjects=[]
runs=[]
wheel_tap_stamps=[]
for idx, row in sync.iterrows():

    subjects.append(row['SubjectId'])
    runs.append(row['RunId'])

    wheel_start=row['wheel.start']
    wheel_end=row['wheel.end']
    wheel_stamps=0.01*np.arange((wheel_end-wheel_start)/0.01)+wheel_start
    #print(idx,wheel_stamps[0], wheel_start)
    
    wheel_tap_idx=row['wheel.tap.idx']
    wheel_tap_stamp=wheel_stamps[wheel_tap_idx-1] #assuming the given wheel.tap.idx starts with 1
    #print(wheel_start,wheel_end, wheel_tap_idx,wheel_tap_stamp)
    
    wheel_tap_stamps.append(wheel_tap_stamp)
    #print('wheel tap stamp = {}'.format(datetime.datetime.utcfromtimestamp(wheel_tap_stamp)))


In [10]:
# 2) Subj-Acceleration (gathered_data) data is already in relation to wheel timestamps. 
#    So, find the index which corresponds to the wheel-sync-timestamp value (from above) (gathered_data_sync_index)

indxs=[]
subjects_accls=[]
#temp=[]
for i, row in sync.iterrows():
    
    #subject_id=row['SubjectId']
    #run_id=row['RunId']
    wheel_start=row['wheel.start']


    # Gather subject raw accls data from raw.h5 
    path_to_subject=row["subject.path"]
    subject_data=raw.get(path_to_subject)
    subject_data=list(subject_data)
    subject_data=np.array(subject_data).T
    subjects_accls.append(subject_data)


    # Generate corrected stamps corresponding to subject sensor entries
    subject_stamps=0.01*np.arange(len(subject_data))+wheel_start # Note wheel.start=subject.start-subject.delta as verified in sync using excel, so this NB is inline with previous ones
    #temp.append(subject_stamps)
    subject_idx=np.argwhere(subject_stamps==wheel_tap_stamps[i])[0][0]
    indxs.append(subject_idx)
    #print(i,len(subject_stamps), wheel_tap_stamps[i], subject_idx)
 


In [11]:
# 3. Create another timestamp that puts the sync.start time from annotation.csv into the place of the gathered_data_sync_index 
# and then creates the full time stamp based on 100 hz


data=[]

# Pull start time stamps corresponding to Sync events (note: walks are properly organized in annotation.csv)
sync_annot_stamps=annotation[annotation['Annotation']=='Sync']['start'].values

for i, accls_set  in enumerate(subjects_accls):
    stamps=np.empty(len(accls_set))
    stamps[indxs[i]]=sync_annot_stamps[i] #Assign the stamp corresponding to the start of Sync event to the cell indexed earlier in accordance with wheel tap event
    stamps[indxs[i]:]=sync_annot_stamps[i]+np.arange(len(stamps)-indxs[i])/100 # Complete lagging cells with 0.01 increment
    stamps[0:indxs[i]]=sync_annot_stamps[i]-0.01-(np.arange(indxs[i])/100)[::-1] # Complete leading cells with -0.01 increment # Note: consider dropping all leading cells 

    subject_ids=np.empty(len(stamps), dtype=object)
    subject_ids[:]=subjects[i]
    run_ids=np.empty(len(stamps), dtype=object)
    run_ids[:]=runs[i]

    arr=np.concatenate([stamps.reshape(-1,1),subject_ids.reshape(-1,1),run_ids.reshape(-1,1),accls_set[:,0:-1]], axis=1)

    # lookup average speed data from wheel.csv based on SubjectId, RunId, and nearest number of wheel MID time to corrected subject start time to stamp
    run_wheel_data=wheel.loc[(wheel['SubjectId']==subjects[i] ) & (wheel['RunId']==runs[i])]
    #first_speed_stamp=run_wheel_data["start"].iloc[0]
    #last_speed_stamp=run_wheel_data["end"].iloc[-1]
    speeds=np.zeros_like(stamps)
    for j in range(len(stamps)):   
        time_idx=np.argmin(np.abs(run_wheel_data['mid'].values-stamps[j]))
        speeds[j]=run_wheel_data['speed'].values[time_idx]
    
    arr=np.concatenate([arr,speeds.reshape(-1,1)],axis=1)
    

    # lookup annotation data from annotation.csv based on SubjectId, RunId, and nearest number of annotation START time to stamp 
    annotations=[]
    annotation_types=[]
    run_annot_data=annotation.loc[(annotation['SubjectId']==subjects[i] ) & (annotation['RunId']==runs[i])]
    first_annot_stamp=run_annot_data["start"].iloc[0]
    last_annot_stamp=run_annot_data["end"].iloc[-1]
    for k in range(len(stamps)):   
        time_idx=np.argmin(np.abs(run_annot_data['start'].values-stamps[k]))
        annotations.append(run_annot_data['Annotation'].values[time_idx])
        annotation_types.append(run_annot_data['AnnotationType'].values[time_idx])
    
    annotations=np.array(annotations)
    annotation_types=np.array(annotation_types)

    arr=np.concatenate([arr,annotations.reshape(-1,1),annotation_types.reshape(-1,1)], axis=1)

    arr=arr[indxs[i]:, :] #IGNORE SAMPLES BEFORE SYNC event

    data.append(arr)

In [12]:
isExist = os.path.exists("../data/processed")
if not isExist:
  os.makedirs("../data/processed")

data=np.concatenate(data)
data_columns=["stamp","SubjectId","RunId","x","y","z","speed", "Annotation","AnnotationType"]
data=pd.DataFrame(data, columns=data_columns)
data.to_csv("../data/processed/{}annot_gathered.csv".format(ref), index=False) 

In [13]:
# temp_df=pd.DataFrame(temp[26])
# temp_df.to_csv('temp.csv')
# print(wheel_tap_stamps[26])
# print(datetime.datetime.utcfromtimestamp(wheel_tap_stamps[26]))